# Image Captioning

## 1 Προετοιμασία των δεδομένων

In [1]:
import tensorflow as tf
import keras
import numpy as np

print('tensorflow version:', tf.__version__)
print('keras version:', keras.__version__)
print('numpy version:', np.__version__)

tensorflow version: 1.15.0
keras version: 2.3.1
numpy version: 1.19.2


Using TensorFlow backend.


In [2]:
# Download dataset from https://www.kaggle.com/adityajn105/flickr8k/
img_dir = 'Images'
text_filename = 'captions.txt'

### 1.1 Προετοιμασία των εικόνων

In [3]:
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
import pickle
import os
from time import time

np.random.seed(0)

t0 = time()

model = VGG19()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

print(model.summary())

features = {}

for name in os.listdir(img_dir):
    filename = os.path.join(img_dir, name)
    img = load_img(filename, target_size=(224, 224))
    img = img_to_array(img)
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    img = preprocess_input(img)
    img_id = name.split('.')[0]
    features[img_id] = model.predict(img)

print('len(features) =', len(features))
    
pickle.dump(features, open('features.pkl', 'wb'))

print('t = {:.1f} min'.format((time() - t0)/60))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
___________________________________________

### 1.2 Προετοιμασία του κειμένου